# Imports

In [1]:
import requests
import os
import sys
import web3
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3 import Web3, EthereumTesterProvider
from web3.middleware import construct_sign_and_send_raw_middleware

import solcx

# Functions

In [2]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()
    return solcx.compile_source(source, output_values=["abi", "bin", "bin-runtime"])

# Deploy token to Goerli

Set-up infura.

In [3]:
# Get the Infura key.
infura_key=os.environ["INFURA_KEY"]

In [4]:
# Connect with Infura.
infura_url = f"https://goerli.infura.io/v3/{infura_key}"

w3 = web3.Web3(web3.Web3.HTTPProvider(infura_url))
print("isConnected=", w3.isConnected())

isConnected= True


Set-up the account.

In [5]:
account_addr = "0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B"
private_key = os.environ["PRIVATE_KEY"]

account: LocalAccount = Account.from_key(private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

Compile the contract & deploy.

In [7]:
interface = compile_source_file("contracts/SorrentumToken.sol")
interface = interface["<stdin>:SorrentumToken"]

In [8]:
pre_mint = 5_000
max_supply = 30_000
name = "Sorrentum"
symbol = "SORR"
func = w3.eth.contract(
    abi=interface["abi"],
    bytecode=interface["bin"]).constructor(name, symbol, pre_mint, max_supply)
# Deploy the token to Goerli.
tx_hash = func.transact({"from": account_addr})

In [12]:
tx_hash

HexBytes('0x1ca82f08b3c520906f7e96e167e97404101b4411239736bc2e03846317b02992')

In [13]:
reciept = w3.eth.get_transaction_receipt(tx_hash)

In [14]:
reciept

AttributeDict({'blockHash': HexBytes('0xeaf9d2af77a780812c732d76c29122c76209ac571706885e3a8dbff7813e7e91'),
 'blockNumber': 8577544,
 'contractAddress': '0x14E3Cd22bAF93aAE4f8F576fD8Fb918f1016bB81',
 'cumulativeGasUsed': 16936144,
 'effectiveGasPrice': 273478380915,
 'from': '0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B',
 'gasUsed': 1878199,
 'logs': [AttributeDict({'address': '0x14E3Cd22bAF93aAE4f8F576fD8Fb918f1016bB81',
   'blockHash': HexBytes('0xeaf9d2af77a780812c732d76c29122c76209ac571706885e3a8dbff7813e7e91'),
   'blockNumber': 8577544,
   'data': '0x',
   'logIndex': 167,
   'removed': False,
   'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x0000000000000000000000006531e2613bbbeecd898356f7b9befbaefd42804b')],
   'transactionHash': HexBytes('0x1ca82f08b3c520906f7e96e167e97404101b4411239736bc2e03846317b02992'),
   'transactionIndex': 138}),

# Get Deployment Price

In [17]:
gas_used = 1_878_199 
gas_price_gwei = 23
eth_price = 1_654

In [19]:
total_gwei = gas_used * gas_price_gwei
total_eth = total_gwei * 0.000000001

In [26]:
total_gwei

43198577

In [27]:
total_eth

0.043198577

In [21]:
total_eth * eth_price

71.45044635800001

# Check the balance

In [11]:
reciept["contractAddress"]

'0x14E3Cd22bAF93aAE4f8F576fD8Fb918f1016bB81'

In [22]:
surr_token = w3.eth.contract(address=reciept["contractAddress"], abi=interface["abi"])

In [24]:
balance = surr_token.functions.balanceOf(account_addr).call()
balance

5000000000000000000000

Check if the balance matches the expected minted amount.

In [25]:
balance == pre_mint * 10**18

True